In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [2]:
def objectiveFunction(x):
    return np.sum(x*2) + 25 * np.sum(np.sin(x)*2)

In [ ]:
def selection(X, scores, k):
    parents = X[(-scores).argsort()[:k]]
    ix = np.arange(k)
    np.random.shuffle(ix)
    n = X.shape[1]
    segment_p1 = np.ones((1, n))
    segment_p2 = np.zeros((1, n))
    start = np.random.randint(n // 2)
    end = start + n // 2
    segment_p1[:, start:end] = 0
    segment_p2[:, start:end] = 1
    offprings = segment_p1 * parents[ix[0], :].reshape((1, n)) + segment_p2 * parents[ix[1], :].reshape((1, n))
    p1, p2 = 1, 2

    for i in range(1, X.shape[0] - k):
        offprings = np.concatenate((offprings,
                                    segment_p1 * parents[ix[p1], :].reshape((1, n)) + segment_p2 * parents[ix[p2], :].reshape((1, n))))
        segment_p1 = np.ones((1, n))
        segment_p2 = np.zeros((1, n))
        start = np.random.randint(n // 2)
        end = start + n // 2
        segment_p1[:, start:end] = 0
        segment_p2[:, start:end] = 1
        p1 = p1 + 1 if p1 < k - 1 else 0
        p2 = p2 + 1 if p2 < k - 1 else 0
        np.random.shuffle(ix)

    return parents, offprings


In [4]:
def mutation(offprings, p_mut):
    filter = np.random.binomial(1, p_mut, size=(offprings.shape[0], offprings.shape[1]))
    newOffprings = offprings + filter * np.random.randn(offprings.shape[0], offprings.shape[1])
    return newOffprings

In [5]:
def evaluation(X, penalty):
    N, n = X.shape
    scores = np.zeros(N)

    for i in range(N):
        constraintCounter = 0

        if np.sum(X[i, :] > 10) > 0:
            constraintCounter += 1
        if np.sum(X[i, :] < 0) > 0:
            constraintCounter += 1
        if X[i, :].prod() < 0.75:
            constraintCounter += 1
        if X[i, :].sum() > 7.5 * n:
            constraintCounter += 1
        scores[i] = objectiveFunction(X[i, :]) + penalty * constraintCounter

    return scores

In [6]:
N = 20
n = 2
X = -2 * np.pi + 4 * np.pi * np.random.rand(N, n)
p_cross = 0.7
k = int(p_cross * N)
p_mut = 0.05
maxGenerations = 4000
penalty = 0.6

for t in range(maxGenerations):
    scores = evaluation(X, penalty)
    print('generation {}, best subject score {:.10f}'.format(t, scores.max()))
    parents, offsprings = selection(X, scores, k)
    offsprings = mutation(offsprings, p_mut)
    X = np.concatenate((parents, offsprings))

scores = evaluation(X, penalty)
print('solution')
print(X[scores.argmax(), :])

generation 0, best subject score 64.0774247862
generation 1, best subject score 87.8719161152
generation 2, best subject score 87.8719161152
generation 3, best subject score 90.4889039779
generation 4, best subject score 90.4889039779
generation 5, best subject score 90.4889039779
generation 6, best subject score 90.4889039779
generation 7, best subject score 99.2143769273
generation 8, best subject score 99.2143769273
generation 9, best subject score 99.2143769273
generation 10, best subject score 99.2143769273
generation 11, best subject score 99.2143769273
generation 12, best subject score 99.2143769273
generation 13, best subject score 99.2143769273
generation 14, best subject score 99.2143769273
generation 15, best subject score 104.3561135518
generation 16, best subject score 104.3561135518
generation 17, best subject score 104.3561135518
generation 18, best subject score 104.3561135518
generation 19, best subject score 104.3561135518
generation 20, best subject score 105.4010310

In [7]:
scores.max()

np.float64(106.36315258575439)

In [8]:
X[scores.argmax(), :].sum()

np.float64(3.2201713680677213)

In [9]:
X[scores.argmax(), :].prod()

np.float64(2.59237556168467)

In [10]:
7.5 * n

15.0